In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate , Dropout , LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError

In [2]:
import pandas as pd
df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/dfn3share.csv')
dffull = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/taskfullshare.csv')

In [3]:
# num_users = df.user.nunique()
# num_services = df.service.nunique()
# num_categories=df.subcat.nunique()
num_users = 1514100
num_services = 589
num_categories = 81

embedding_dim=50
num_genders=4
learning_rate=0.005
batch_size=32
num_epochs=5

In [4]:
df=df.iloc[:,1:6]

In [5]:
#@title Model
# Define the inputs
user_id_input = Input(shape=(1,), name='user_id')
gender_input = Input(shape=(1,), name='gender')
service_id_input = Input(shape=(1,), name='service_id')
category_input = Input(shape=(1,), name='category')
# time_input = Input(shape=(1,), name='time')

# Define the embedding layers
user_id_emb = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_id_emb')(user_id_input)
gender_emb = Embedding(input_dim=num_genders, output_dim=embedding_dim, name='gender_emb')(gender_input)
service_id_emb = Embedding(input_dim=num_services, output_dim=embedding_dim, name='service_id_emb')(service_id_input)
category_emb = Embedding(input_dim=num_categories, output_dim=embedding_dim, name='category_emb')(category_input)

# Concatenate the embeddings
concatenated = Concatenate()([user_id_emb, gender_emb, service_id_emb, category_emb])
x = Dropout(0.2)(concatenated)
x = LayerNormalization()(x)
# Define the transformer model
# transformer_layer = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=embedding_dim)
# transformer_output = transformer_layer(concatenated, concatenated)
# transformer_output = tf.keras.layers.GlobalAveragePooling1D()(transformer_output)
# Add multi-head attention layer
# x = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(x)

transformer_layer = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=embedding_dim)
x = transformer_layer(x, x)

x = Dropout(0.2)(x)
x = LayerNormalization()(x)

# Add feed-forward network
x = Dense(units=embedding_dim, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=1, activation='linear')(x)
# Define the output layer
output = x
# output_layer = Dense(1, activation=None)


# Create the model
model = Model(inputs=[user_id_input, gender_input, service_id_input, category_input], outputs=[output])

# Compile the model
optimizer = Adam(lr=learning_rate)
loss = 'mean_squared_error'
metrics = [MeanSquaredError()]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
df.columns

Index(['user', 'service', 'count', 'subcat', 'gender'], dtype='object')

In [6]:
# Define the training data
train_data = ([df['user'].values, df['gender'].values, df['service'].values, df['subcat'].values],df['count'].values)

In [7]:
# Train the model
history = model.fit(train_data[0], train_data[1], batch_size=batch_size, epochs=2, validation_split=0.2)
model.save("/content/drive/MyDrive/Colab Notebooks/sanjagh/Recommand/savedmodels/model02-Trans")

Epoch 1/2
28594/28594 [==============================] - 760s 26ms/step - loss: 1.5266 - mean_squared_error: 1.5266 - val_loss: 0.7265 - val_mean_squared_error: 0.7265
Epoch 2/2
28594/28594 [==============================] - 768s 27ms/step - loss: 1.5150 - mean_squared_error: 1.5150 - val_loss: 0.7266 - val_mean_squared_error: 0.7266


In [10]:
df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/dfn3share.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
dfuser=df[df.user==35]
# dfuserservice = pd.merge(dfuser, servicesdf, on="service")
pservice=set(df.sort_values('count' , ascending= False).head(100).service.values)
dfp=pd.DataFrame(columns=['user',	'service'	,'subcat',  'gender'])
for u in set(dfuser.user.values):
  for s in pservice:
    dfp1=pd.DataFrame({'user' : [(u)],	'service':[(s)] ,'subcat':df[df.service == s].iloc[0,3] ,
                       'gender' : dfuser[dfuser.user == u ].iloc[0,4]})
    dfp = pd.concat([dfp, dfp1], ignore_index = True)
ratings = model.predict( (tf.convert_to_tensor(dfp['user'].values, dtype=tf.int64) ,
                               tf.convert_to_tensor(dfp['gender'].values, dtype=tf.int64),
                               tf.convert_to_tensor( dfp['service'].values, dtype=tf.int64), 
                               tf.convert_to_tensor(dfp['subcat'].values, dtype=tf.int64)))
ratings

1/1 [==============================] - 0s 252ms/step


array([[[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]],

       [[1.4725496]]], dtype=float32)

### another Example

In [ ]:
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
ZipFile("movielens.zip", "r").extractall()

In [ ]:
# users = pd.read_csv(
#     "ml-1m/users.dat",
#     sep="::",
#     names=["user_id", "sex", "age_group", "occupation", "zip_code"],
# )

# ratings = pd.read_csv(
#     "ml-1m/ratings.dat",
#     sep="::",
#     names=["user_id", "movie_id", "rating", "unix_timestamp"],
# )

movies = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/movies", names=["movie_id", "title", "genres"]
)
movies

,movie_id,title,genres
NaN,1,Toy Story (1995),Animation|Children's|Comedy
0.0,2,Jumanji (1995),Adventure|Children's|Fantasy
1.0,3,Grumpier Old Men (1995),Comedy|Romance
2.0,4,Waiting to Exhale (1995),Comedy|Drama
3.0,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3877.0,3948,Meet the Parents (2000),Comedy
3878.0,3949,Requiem for a Dream (2000),Drama
3879.0,3950,Tigerland (2000),Drama
3880.0,3951,Two Family House (2000),Drama


In [ ]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

In [ ]:
movies

,movie_id,title,genres
NaN,movie_1,Toy Story (1995),Animation|Children's|Comedy
0.0,movie_2,Jumanji (1995),Adventure|Children's|Fantasy
1.0,movie_3,Grumpier Old Men (1995),Comedy|Romance
2.0,movie_4,Waiting to Exhale (1995),Comedy|Drama
3.0,movie_5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3877.0,movie_3948,Meet the Parents (2000),Comedy
3878.0,movie_3949,Requiem for a Dream (2000),Drama
3879.0,movie_3950,Tigerland (2000),Drama
3880.0,movie_3951,Two Family House (2000),Drama


In [ ]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for genre in genres:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|"))
    )

In [ ]:
movies.head(1)

,movie_id,title,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
NaN,movie_1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    }
)

In [ ]:
ratings_data.head(2)

,user_id,movie_ids,ratings,timestamps
0,user_user_1,"[movie_movie_3186, movie_movie_1721, movie_mov...","[4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, ...","[978300019, 978300055, 978300055, 978300055, 9..."
1,user_user_10,"[movie_movie_597, movie_movie_858, movie_movie...","[4.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 5.0, 3.0, ...","[978224375, 978224375, 978224375, 978224400, 9..."


In [ ]:
ratings.head(1)

,user_id,movie_id,rating,unix_timestamp
0,user_user_1,movie_movie_1193,5.0,978300760


In [ ]:
sequence_length = 4
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["timestamps"]

In [ ]:
ratings_data.head(1)

,user_id,movie_ids,ratings
0,user_user_1,"[[movie_movie_3186, movie_movie_1721, movie_mo...","[[4.0, 4.0, 5.0, 5.0], [5.0, 5.0, 3.0, 5.0], [..."


In [ ]:
ratings_data.iloc[0,1]

In [ ]:
ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed.movie_ids = ratings_data_transformed.movie_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

In [ ]:
ratings_data_transformed

,user_id,sequence_movie_ids,sequence_ratings,sex,age_group,occupation
0,user_user_1,"movie_movie_3186,movie_movie_1721,movie_movie_...","4.0,4.0,5.0,5.0",F,group_group_1,occupation_occupation_10
1,user_user_1,"movie_movie_1270,movie_movie_1022,movie_movie_...","5.0,5.0,3.0,5.0",F,group_group_1,occupation_occupation_10
2,user_user_1,"movie_movie_2340,movie_movie_1836,movie_movie_...","3.0,5.0,4.0,4.0",F,group_group_1,occupation_occupation_10
3,user_user_1,"movie_movie_3408,movie_movie_1207,movie_movie_...","4.0,4.0,5.0,4.0",F,group_group_1,occupation_occupation_10
4,user_user_1,"movie_movie_2804,movie_movie_260,movie_movie_7...","5.0,4.0,3.0,5.0",F,group_group_1,occupation_occupation_10
...,...,...,...,...,...,...
498618,user_user_999,"movie_movie_2676,movie_movie_2540,movie_movie_...","3.0,2.0,3.0,3.0",M,group_group_25,occupation_occupation_15
498619,user_user_999,"movie_movie_1363,movie_movie_765,movie_movie_3...","3.0,3.0,4.0,2.0",M,group_group_25,occupation_occupation_15
498620,user_user_999,"movie_movie_3565,movie_movie_1410,movie_movie_...","4.0,2.0,3.0,3.0",M,group_group_25,occupation_occupation_15
498621,user_user_999,"movie_movie_2269,movie_movie_2504,movie_movie_...","3.0,3.0,2.0,2.0",M,group_group_25,occupation_occupation_15


In [ ]:
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)

In [ ]:
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}

USER_FEATURES = ["sex", "age_group", "occupation"]

MOVIE_FEATURES = ["genres"]

In [ ]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # The last movie id in the sequence is the target movie.
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

In [ ]:
def create_model_inputs():
    return {
        "user_id": layers.Input(name="user_id", shape=(1,), dtype=tf.string),
        "sequence_movie_ids": layers.Input(
            name="sequence_movie_ids", shape=(sequence_length - 1,), dtype=tf.string
        ),
        "target_movie_id": layers.Input(
            name="target_movie_id", shape=(1,), dtype=tf.string
        ),
        "sequence_ratings": layers.Input(
            name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
        ),
        "sex": layers.Input(name="sex", shape=(1,), dtype=tf.string),
        "age_group": layers.Input(name="age_group", shape=(1,), dtype=tf.string),
        "occupation": layers.Input(name="occupation", shape=(1,), dtype=tf.string),
    }

In [ ]:
def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_movie_features=True,
):

    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices.
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions.
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations.
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create a movie embedding encoder
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(math.sqrt(len(movie_vocabulary)))
    # Create a lookup to convert string values to integer indices.
    movie_index_lookup = StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup",
    )
    # Create an embedding layer with the specified dimensions.
    movie_embedding_encoder = layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        name=f"movie_embedding",
    )
    # Create a vector lookup for movie genres.
    genre_vectors = movies[genres].to_numpy()
    movie_genres_lookup = layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )
    # Create a processing layer for genres.
    movie_embedding_processor = layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## Define a function to encode a given movie id.
    def encode_movie(movie_id):
        # Convert the string input values into integer indices.
        movie_idx = movie_index_lookup(movie_id)
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding
        if include_movie_features:
            movie_genres_vector = movie_genres_lookup(movie_idx)
            encoded_movie = movie_embedding_processor(
                layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## Encoding target_movie_id
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## Encoding sequence movie_ids.
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)
    # Add the positional encoding to the movie encodings and multiply them by rating.
    encoded_sequence_movies_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs.
    for encoded_movie in tf.unstack(
        encoded_sequence_movies_with_poistion_and_rating, axis=1
    ):
        encoded_transformer_features.append(tf.expand_dims(encoded_movie, 1))
    encoded_transformer_features.append(encoded_target_movie)

    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )

    return encoded_transformer_features, encoded_other_features

In [ ]:
include_user_id = False
include_user_features = False
include_movie_features = False

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # Create a multi-headed attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

/usr/local/lib/python3.9/dist-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5)

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
3725-180-30=3300 
تعاون 3600 3900 3500